In [1]:
import pandas as pd
import numpy as np

In [3]:
energy_info = pd.read_csv(f"../data/energy_info.csv")

In [4]:
energy_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3509652 entries, 0 to 3509651
Data columns (total 25 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   Unnamed: 0               int64  
 1   LCLid                    object 
 2   Acorn                    object 
 3   Acorn_id                 int64  
 4   energy_sum               float64
 5   date                     object 
 6   temperatureMax           float64
 7   windBearing              int64  
 8   dewPoint                 float64
 9   cloudCover               float64
 10  windSpeed                float64
 11  pressure                 float64
 12  apparentTemperatureHigh  float64
 13  visibility               float64
 14  humidity                 float64
 15  apparentTemperatureLow   float64
 16  apparentTemperatureMax   float64
 17  uvIndex                  float64
 18  sunsetTime               object 
 19  temperatureLow           float64
 20  temperatureMin           float64
 21  temperat

In [ ]:
def extract_date(df, date_col_name):
    df[f'{date_col_name}_year'] = df[date_col_name].dt.year
    df[f'{date_col_name}_month'] = df[date_col_name].dt.month
    df[f'{date_col_name}_day'] = df[date_col_name].dt.day
    df[f'{date_col_name}_day_of_week'] = df[date_col_name].dt.dayofweek
    df[f'{date_col_name}_day_of_week_name'] = df[date_col_name].dt.day_name()
    df[f'{date_col_name}_is_weekend'] = np.where(df[f'{date_col_name}_day_of_week_name'].isin(['Sunday', 'Saturday']), 1, 0)
    df = df.drop([f'{date_col_name}_day_of_week_name'], axis=1)
    
